# Understanding this weird dataset.

In [6]:
import pandas as pd

In [32]:
# print("Original dataset shapes:")
# print(f"Traffic data (V): {dataset.shape}")
# print(f"Adjacency matrix (W): {dataset2.shape}")
# print()

# small_traffic = dataset.iloc[:, :4].copy()

# small_adj = np.array([
#     [0.0, 1.0, 0.5, 0.0],  # Node 0 connects to 1 and weakly to 2
#     [1.0, 0.0, 1.0, 0.5],  # Node 1 connects to 0,2 and weakly to 3
#     [0.5, 1.0, 0.0, 1.0],  # Node 2 connects to 1,3 and weakly to 0
#     [0.0, 0.5, 1.0, 0.0]   # Node 3 connects to 2 and weakly to 1
# ])

# # Save the small dataset
# small_traffic.to_csv("../dataset/V_small_4.csv", index=False, header=False)
# np.savetxt("../dataset/W_small_4.csv", small_adj, delimiter=",", fmt='%.1f')

# print("Small dataset created:")
# print(f"Traffic data: {small_traffic.shape}")
# print(f"Adjacency matrix: {small_adj.shape}")
# print()
# print("Sample traffic data (first 5 timesteps):")
# print(small_traffic.head())
# print()
# print("Adjacency matrix:")
# print(small_adj)

Original dataset shapes:
Traffic data (V): (12671, 228)
Adjacency matrix (W): (227, 228)

Small dataset created:
Traffic data: (12671, 4)
Adjacency matrix: (4, 4)

Sample traffic data (first 5 timesteps):
   7.109999999999999432e+01  6.600000000000000000e+01  \
0                      68.1                      66.8   
1                      68.0                      64.3   
2                      68.3                      67.8   
3                      68.9                      69.5   
4                      66.6                      69.1   

   6.459999999999999432e+01  6.559999999999999432e+01  
0                      61.7                      66.7  
1                      66.6                      68.7  
2                      65.9                      66.6  
3                      61.2                      67.4  
4                      65.1                      65.2  

Adjacency matrix:
[[0.  1.  0.5 0. ]
 [1.  0.  1.  0.5]
 [0.5 1.  0.  1. ]
 [0.  0.5 1.  0. ]]


In [36]:
dataset = pd.read_csv(r"../dataset/V_small_4.csv")
dataset2 = pd.read_csv(r"../dataset/W_small_4.csv")

In [37]:
# === STGCN DataLoader Preparation ===
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Convert main traffic DataFrame to numpy: [time_steps, num_nodes]
data_np = dataset.values

# Define historical and prediction window lengths
n_his, n_pred = 12, 3  # e.g., use past 12 steps to predict next 3

class STGCNDataset(Dataset):
    def __init__(self, data, his_len, pred_len):
        self.data = data
        self.his_len = his_len
        self.pred_len = pred_len
        self.num_samples = data.shape[0] - his_len - pred_len + 1

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # input: [his_len, num_nodes]
        x = self.data[idx : idx + self.his_len]
        # target: [pred_len, num_nodes]
        y = self.data[idx + self.his_len : idx + self.his_len + self.pred_len]
        return torch.from_numpy(x).float(), torch.from_numpy(y).float()

# Instantiate dataset and loader
stgcn_dataset = STGCNDataset(data_np, n_his, n_pred)
stgcn_loader = DataLoader(stgcn_dataset, batch_size=64, shuffle=True)

# Load adjacency matrix (defines graph connectivity)
w = dataset2.values
adj = torch.from_numpy(w).float()  # shape: [num_nodes, num_nodes]

# Inspect shapes
print(f"Dataset samples: {len(stgcn_dataset)}")
print(f"Batch X shape: {next(iter(stgcn_loader))[0].shape}")  # [batch, his_len, num_nodes]
print(f"Adjacency shape: {adj.shape}")

Dataset samples: 12656
Batch X shape: torch.Size([64, 12, 4])
Adjacency shape: torch.Size([3, 4])
